# Importing Libraries

In [1]:
import tensorflow as tf

2023-12-08 15:16:19.994895: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
print(tf.__version__)

2.5.0


In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate

In [4]:
import sys
sys.path.append('../../_EXPERIMENTS')

In [5]:
#import cactas as C
import os
import random
import numpy as np
import nrrd

In [6]:
gpu_available = tf.test.is_gpu_available()
gpu_available

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-12-08 15:16:26.271796: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 15:16:26.275225: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-12-08 15:16:26.441193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-12-08 15:16:26.441230: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-12-08 15:16:26.463355: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] 

True

In [7]:
import random
from tensorflow.keras.optimizers import Adam


In [8]:
from tensorflow.keras import backend as K

# Defining Functions

In [9]:
def load_padded_images(IMG_DATAPATH):
    directory = IMG_DATAPATH
    min_filesize_bytes = 15728640 #15MB
    images = []
    for filename in os.listdir(directory):
        if (filename.startswith(".ipynb_checkpoints")):
            print()
        else:
            images.append(filename)
    return images

In [10]:
def load_padded_labels(LAB_DATAPATH):
    directory = LAB_DATAPATH
    min_filesize_bytes = 15728640 #15MB
    labels = []
    for filename in os.listdir(directory):
        if (filename.startswith(".ipynb_checkpoints")):
            print()
        else:
            labels.append(filename)
    return labels

In [11]:
def buildModel(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool2)

    up4 = concatenate([UpSampling3D(size=(2, 2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(up4)

    up5 = concatenate([UpSampling3D(size=(2, 2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up5)

    conv6 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv5)

    model = Model(inputs=[inputs], outputs=[conv6])

    return model

In [12]:
def split_data(images, labels, split_ratio=0.8):
    
    data = list(zip(images, labels))
    
    random.shuffle(data)

  # Get the split index
    split_index = int(len(data) * split_ratio)

  # Split data into train and test
    train_data = data[:split_index]
    test_data = data[split_index:]

  # Separate into images and labels
    X_train, y_train = zip(*train_data) 
    X_test, y_test = zip(*test_data)
   
    return X_train, X_test, y_train, y_test

In [13]:
def batch_normalize(data_path, file_list, batch_size):
    normalized_data = []
    for file in file_list:
        data, header = nrrd.read(data_path + "/" + file)
        normalized_data.append((data - np.min(data)) / (np.max(data) - np.min(data)))

        if len(normalized_data) == batch_size:
            yield normalized_data
            normalized_data = []

    if normalized_data:
        yield normalized_data

In [14]:
def normalization(IMG_DATAPATH, LAB_DATAPATH, X_train, y_train, X_test, y_test, batch_size):
    norm_X_train = []
#    norm_y_train = []
    norm_X_test = []
#    norm_y_test = []

    for batch in batch_normalize(IMG_DATAPATH, X_train, batch_size):
        norm_X_train.extend(batch)
    
#     for batch in batch_normalize(LAB_DATAPATH, y_train, batch_size):
#         norm_y_train.extend(batch)

    for batch in batch_normalize(IMG_DATAPATH, X_test, batch_size):
        norm_X_test.extend(batch)
    
#     for batch in batch_normalize(LAB_DATAPATH, y_test, batch_size):
#         norm_y_test.extend(batch)

    return norm_X_train, norm_X_test

In [15]:
def extract_slices_3d(X_train, y_train, X_test, y_test):
    
    
    slices = []
    for i in range(X_train.shape[0]):
        slice_3d = X_train[i]
        slices.append(slice_3d)
    X_train_array = np.array(slices)

    slices2 = []
    for i in range(X_test.shape[0]):
        slice_3d = X_test[i]
        slices2.append(slice_3d)
    X_test_array = np.array(slices2)

    # Process target data (y_train and y_test)
    new_slices = []
    for i in range(y_train.shape[0]):
        slice_3d = y_train[i]
        slices = np.where(slice_3d != 0, True, False)
        new_slices.append(slices)
    y_train_array = np.array(new_slices)

    new_slices2 = []
    for i in range(y_test.shape[0]):
        slice_3d = y_test[i]
        slices = np.where(slice_3d != 0, True, False)
        new_slices2.append(slices)
    y_test_array = np.array(new_slices2)

    
    print(X_train_array.shape, y_train_array.shape, X_test_array.shape, y_test_array.shape)

    return X_train_array, y_train_array, X_test_array, y_test_array

In [16]:
def parse_y(data_path, y_train, y_test):
    norm_y_train = []
    norm_y_test = []
    for file in y_train:
        data, header = nrrd.read(data_path + "/" + file)
        norm_y_train.append(data)
    
    for file in y_test:
        data, header = nrrd.read(data_path + "/" + file)
        norm_y_test.append(data)
        
    return norm_y_train, norm_y_test

In [17]:
def iou(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

In [18]:
def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred)
    jac = (intersection + 1.) / (union - intersection + 1.)
    return K.mean(jac)

In [19]:
def iou_thresholded(y_true, y_pred, threshold=0.5, smooth=1.):
    y_pred = threshold_binarize(y_pred, threshold)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)


def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Loading Data and Normalizing

In [42]:
IMG_DATAPATH = os.path.join('ChunksESUS')
LAB_DATAPATH = os.path.join('Chunks_Labels_ESUS')

In [43]:
images = load_padded_images(IMG_DATAPATH)
labels = load_padded_labels(LAB_DATAPATH)

In [44]:
len(images)

3264

In [45]:
X_train, X_test, y_train, y_test = split_data(images, labels)

In [46]:
percentage_to_select = 20

num_samples_to_select = int(len(X_train) * percentage_to_select / 100)

random_indices = random.sample(range(len(X_train)), num_samples_to_select)

X_train_selected = [X_train[i] for i in random_indices]
y_train_selected = [y_train[i] for i in random_indices]

num_samples_to_select = int(len(X_test) * percentage_to_select / 100)
random_indices = random.sample(range(len(X_test)), num_samples_to_select)
X_test_selected = [X_test[i] for i in random_indices]
y_test_selected = [y_test[i] for i in random_indices]

In [47]:
len(X_train_selected)

522

In [26]:
X_train, X_test = normalization(IMG_DATAPATH, LAB_DATAPATH, X_train_selected, y_train_selected, X_test_selected, y_test_selected, 2)

/tmp/ipykernel_1228788/1398699592.py:5: RuntimeWarning: invalid value encountered in true_divide
  normalized_data.append((data - np.min(data)) / (np.max(data) - np.min(data)))


In [48]:
y_train, y_test = parse_y(LAB_DATAPATH, y_train_selected, y_test_selected)

In [50]:
np.any(y_train != 0)

True

In [52]:
arr = np.nonzero(y_train)

In [54]:
print(np.shape(arr))

(4, 6697)


In [41]:
X_train.

array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,    

In [28]:
len(X_train)

522

In [29]:
X_train = np.array(X_train)


In [30]:
X_test = np.array(X_test)

In [31]:
X_train.shape

(522, 128, 128, 128)

In [40]:
#X_train = X_train.reshape(128, 128, 384, 1)
#X_test = X_test.reshape(128, 128, 384, 1)

ValueError: cannot reshape array of size 270532608 into shape (128,128,384,1)

In [32]:
y_train = np.array(y_train)

In [33]:
y_test = np.array(y_test)

In [34]:
X_train.shape

(522, 128, 128, 128)

In [73]:
X_train.shape[0]

783

In [51]:
X_train.shape[3]

128

In [52]:
X_train[0].shape

(128, 128, 128)

In [35]:
X_train, y_train, X_test, y_test = extract_slices_3d(X_train, y_train, X_test, y_test)

(522, 128, 128, 128) (522, 128, 128, 128) (130, 128, 128, 128) (130, 128, 128, 128)


In [75]:
y_test.shape

(195, 128, 128, 128)

In [76]:
y_test.dtype

dtype('bool')

In [77]:
len(X_train)

783

In [40]:
X_train.shape

(522, 128, 128, 128)

In [79]:
X_test.shape

(195, 128, 128, 128)

In [53]:
y_train.shape

(52, 128, 128, 128)

In [54]:
true_count = np.count_nonzero(y_train[0])
#false_count = len(X_train[0]) - true_count
false_count = np.count_nonzero(y_train[0] - 1)
print(true_count, false_count)

0 2097152


# Model Building

In [36]:
model = buildModel((128, 128, 128, 1))

2023-12-08 15:20:10.696398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-12-08 15:20:10.704528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-12-08 15:20:10.708660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-12-08 15:20:10.716372: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-12-08 15:20:10.716412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-12-08 15:20:10.716416: I tensorflow/core/common_runtime/gp

In [37]:
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics='accuracy')

/home/dhruvjignesh.shah001/anaconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [38]:
model.fit(X_train, y_train, batch_size=4, epochs=5)

2023-12-08 15:20:39.887270: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-12-08 15:20:39.908746: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245800000 Hz


Epoch 1/5


2023-12-08 15:20:41.627592: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-12-08 15:20:42.613717: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-12-08 15:20:43.670255: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-12-08 15:20:43.670276: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-12-08 15:20:43.670738: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-12-08 15:20:43.784305: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-12-08 15:20:44.841180: I tensorflow/stream_exec

 42/131 [========>.....................] - ETA: 2:52 - loss: nan - accuracy: 0.9998

KeyboardInterrupt: 

# Testing Around

In [14]:
X_train = np.load("X_train.npy")

In [15]:
X_test = np.load("X_test.npy")

In [16]:
y_train = np.load("y_train.npy")

In [17]:
y_test = np.load("y_test.npy")

In [39]:
print(X_train)

[[[[       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   ...
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]]

  [[       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   ...
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]
   [       nan        nan        nan ...        nan        nan
           nan]]

  [[       nan        nan        n

In [10]:
X_train.max()

1.0

In [19]:
X_train.shape

(43, 128, 128, 384)

In [12]:
X_train.min()

0.0

In [20]:
X_train_128 = X_train[:, :, :, :128]

In [21]:
X_train_128 = X_train_128.reshape((43, 128, 128, 128))

In [23]:
y_train_128 = y_train[:, :, :, :128]

In [24]:
y_test_128 = y_test[:, :, :, :128]

In [25]:
X_test_128 = X_test[:, :, :, :128]

In [27]:
y_train_128 = y_train_128.reshape((43, 128, 128, 128))
y_test_128 = y_test_128.reshape((10, 128, 128, 128))
X_test_128 = X_test_128.reshape((10, 128, 128, 128))